In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import time
%matplotlib inline  
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/mu529/bus-Capstone')

# these two modules are homemade
import gtfs
import arrivals
import ttools
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [10]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('2015-12-03','gtfs/')
stops = gtfs.load_stops('2015-12-03','gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('2015-12-03','gtfs/')
tcal=gtfs.TransitCalendar('2015-12-03')
thurs_services = tcal.get_service_ids('2015-12-03')
thurs_trips = trips.service_id.isin(thurs_services)
thurs_stops = stop_times.reset_index().set_index('trip_id').loc[thurs_trips]
print 'Finished loading GTFS data.'

Finished loading GTFS data.


In [99]:
interpolated = pd.read_csv('2015-12-03_interpolated.csv')

In [100]:
interpolated['interpolated_arrival_time'] = pd.to_timedelta(interpolated['interpolated_arrival_time'])

In [102]:
interpolated = interpolated.merge(stop_times[['arrival_time','stop_sequence']],how='left',left_on=['trip_id','stop_id'],right_index=True)

In [103]:
# bools = interpolated.duplicated(subset=['route','trip_id','trip_date','vehicle_id','stop_id'],take_last=False) + interpolated.duplicated(subset=['route','trip_id','trip_date','vehicle_id','stop_id'],take_last=True)
# interpolated[bools].sort(['route','trip_id','trip_date','vehicle_id']).shape

In [104]:
interpolated.set_index(['route','trip_id','trip_date','vehicle_id','stop_sequence','stop_id'],inplace=True)

## Get mean travel times based interpolations between stops

In [105]:
travel_times = interpolated['interpolated_arrival_time']/ttools.datetime.timedelta(seconds=1)

In [106]:
travel_times.head()

route          trip_id                            trip_date   vehicle_id     stop_sequence  stop_id
MTA NYCT_M101  OH_D5-Weekday-SDon-036500_M101_38  2015-12-03  MTA NYCT_5879  2              402805     22448.112401
                                                                             3              405057     22505.545780
                                                                             4              403416     22618.526210
                                                                             5              402809     22645.226045
                                                                             6              402811     22716.819142
Name: interpolated_arrival_time, dtype: float64

In [122]:
# travel_times = travel_times.groupby(level=(0,1,2,3)).diff()
diffs = travel_times.groupby(level=(0,1,2,3)).transform(pd.Series.diff)

In [124]:
diffs.iloc[20:40]

route          trip_id                              trip_date   vehicle_id     stop_sequence  stop_id
MTA NYCT_M101  OH_D5-Weekday-SDon-036500_M101_38    2015-12-03  MTA NYCT_5879  24             803158     141.263272
                                                                               25             402835      79.997546
                                                                               26             404171     100.278116
                                                                               27             402838      70.491499
                                                                               28             803163     112.215689
                                                                               29             403424      53.535580
                                                                               31             402842     121.464761
                                                                               32     

In [125]:
diffs2 = pd.DataFrame(diffs).reset_index()

In [127]:
diffs2['prev_stop'] = diffs2['stop_id'].shift()

In [128]:
diffs2.head()

,route,trip_id,trip_date,vehicle_id,stop_sequence,stop_id,interpolated_arrival_time,prev_stop
0,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,2,402805,NaN,NaN
1,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,3,405057,57.433379,402805
2,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,4,403416,112.980430,405057
3,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,5,402809,26.699835,403416
4,MTA NYCT_M101,OH_D5-Weekday-SDon-036500_M101_38,2015-12-03,MTA NYCT_5879,6,402811,71.593097,402809


In [129]:
diffs2.set_index(['prev_stop','stop_id'],inplace=True)

In [130]:
diffs2['elapsed'] = diffs2['interpolated_arrival_time']

In [131]:
diffs2['elapsed'].mean()

87.471864607713698

In [138]:
diffs2.groupby(level=(0,1))['elapsed'].size().head()

prev_stop  stop_id
100014     104087     81
           403676      1
           502256      1
100017     100018     71
           100056     84
dtype: int64

In [139]:
diffs2.groupby(level=(0,1))['elapsed'].mean().head()

prev_stop  stop_id
100014     104087      90.626629
           403676            NaN
           502256            NaN
100017     100018     104.237867
           100056     223.215262
Name: elapsed, dtype: float64

In [140]:
len(diffs2.groupby(level=(0,1)))

65651

In [ ]:
thurs_stops.groupby(level=(0))['stop_sequence'].diff()

## Get the densest stop_id by shape (trip?)

In [61]:
trip_groups = interpolated.groupby(level=(0,2,5,4)).size()

In [64]:
def series_max(s):
    return s.idxmax()[2]

In [65]:
trip_groups.groupby(level=(0)).apply(series_max)

route
MTA NYCT_B1      300009
MTA NYCT_B11     301173
MTA NYCT_B12     301328
MTA NYCT_B13     301505
MTA NYCT_B14     301598
MTA NYCT_B15     553257
MTA NYCT_B16     301677
MTA NYCT_B17     300853
MTA NYCT_B2      300173
MTA NYCT_B20     301959
MTA NYCT_B24     503993
MTA NYCT_B25     302383
MTA NYCT_B26     302372
MTA NYCT_B3      300266
MTA NYCT_B31     300175
MTA NYCT_B32     305166
MTA NYCT_B35     302692
MTA NYCT_B36     300019
MTA NYCT_B37     301016
MTA NYCT_B38     302948
MTA NYCT_B39     403679
MTA NYCT_B4      300107
MTA NYCT_B41     303295
MTA NYCT_B42     303348
MTA NYCT_B43     303711
MTA NYCT_B44     303483
MTA NYCT_B44+    303950
MTA NYCT_B45     303531
MTA NYCT_B46     303592
MTA NYCT_B47     303191
                  ...  
MTABC_Q50        100294
MTABC_Q52        550070
MTABC_Q53        550070
MTABC_Q60        504399
MTABC_Q64        551812
MTABC_Q65        550032
MTABC_Q66        551894
MTABC_Q67        503832
MTABC_Q69        550010
MTABC_Q70        504542
MTABC_Q72 